In [1]:
%matplotlib inline
import sys, os, time
# Select GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
# from tensorflow.keras.applications.mobilenet import preprocess_input, MobileNet
# from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [92]:
BASE_PATH = "/home/ailie/Repos/BBAttacks/data/"

In [3]:
IMAGES_PATH = os.path.join(BASE_PATH, "x_val_0_10000.npy")
LABELS_PATH = os.path.join(BASE_PATH, "y_val.npy")
SYNSET_WORDS_PATH = os.path.join(BASE_PATH, "synset_words.txt")

In [11]:
keras_idx_to_name = {}
f = open(SYNSET_WORDS_PATH,"r")
idx = 0
for line in f:
    parts = line.split(" ")
    keras_idx_to_name[idx] = " ".join(parts[1:])
    idx += 1
f.close()

In [4]:
x_val_raw = np.load(IMAGES_PATH) # loaded as RGB
# didn't upload imagenet data here as it is too large, but it is publicl available
# and the experiments are reproducible 
x_val = preprocess_input(x_val_raw.copy()) # converted to BGR

In [5]:
y_val = np.load(LABELS_PATH)

In [6]:
x_val2 = x_val[:1000].copy()
y_val2 = y_val[:1000].copy()
x_val_raw2 = x_val_raw.copy()

In [7]:
del(x_val)
del(y_val)
del(x_val_raw)

In [8]:
import gc
gc.collect()

22

In [9]:
x_val = x_val2
y_val = y_val2
x_val_raw = x_val_raw2

In [10]:
y_val_one_hot = to_categorical(y_val, 1000)

In [13]:
K.clear_session()
# model = MobileNet()
model = InceptionV3(include_top=True, weights='imagenet')

2022-09-10 04:50:45.893769: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-09-10 04:50:47.544440: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-09-10 04:50:47.544777: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-09-10 04:50:47.547293: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-09-10 04:50:47.549509: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-09-10 04:50:47.549887: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcu

In [14]:
RANDOM_SAMPLE_SIZE = 100
RANDOM_SEED = 40

In [15]:
import random
random.seed(RANDOM_SEED)
sample_indices = random.choices(range(len(x_val)), k=RANDOM_SAMPLE_SIZE)

In [ ]:
plt.imshow(x_val_raw[458]/255)

In [16]:
# sample_indices = range(500)
y_val_one_hot_sample = y_val_one_hot[sample_indices]
x_val_sample = x_val[sample_indices]
x_val_raw_sample = x_val_raw[sample_indices]

In [17]:
y_pred_sample = model.predict(x_val_sample, verbose=1)

2022-09-10 04:50:53.285543: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-09-10 04:50:53.407231: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


4/4 [==============================] - 0s 86ms/step


In [12]:
def top_k_accuracy(y_true, y_pred, k=1):
    '''From: https://github.com/chainer/chainer/issues/606
    
    Expects both y_true and y_pred to be one-hot encoded.
    '''
    argsorted_y = np.argsort(y_pred)[:,-k:]
    return np.any(argsorted_y.T == y_true.argmax(axis=1), axis=0).mean()

In [20]:
top_k_accuracy(y_val_one_hot_sample, y_pred_sample, k=1)

0.67

In [21]:
top_k_accuracy(y_val_one_hot_sample, y_pred_sample, k=5)

0.9

In [22]:
indices = []
agreements = np.argmax(y_pred_sample, axis=1) == np.argmax(y_val_one_hot_sample, axis=1)
for x in range(len(agreements)):
    if agreements[x]:
        indices.append(x)

In [23]:
# y_val_one_hot_sample = y_val_one_hot_sample[indices][:100]
# x_val_sample = x_val_sample[indices][:100]
# x_val_raw_sample = x_val_raw_sample[indices][:100]

In [24]:
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, Dense, Flatten, BatchNormalization
import importlib
import tensorflow as tf

In [66]:
import sys
sys.path.append("/home/ailie/Repos/BBAttacks/attacks/")
sys.path.append("/home/ailie/Repos/BBAttacks/utils/")

import utils
from data_manager import load_data

# Black Box Attacks
import random_noise
import EvoStrategyUniformProbs
import SimbaWrapper

<h3>Random attack</h3>

<h4>Debugging the random attack</h4>

In [67]:
# agreements_cell_test

In [68]:
# np.argmax(y_pred_sample, axis=1) == np.argmax(y_val_one_hot_sample, axis=1)

In [69]:
# plt.imshow(saved_noisy_imgs_cell_test[0])
# print("correct label:", y_val[0])
# print("after:", np.argmax(model.predict(np.expand_dims(saved_noisy_imgs_cell_test[0],0)), axis=1)[0])
# print("initially:", np.argmax(model.predict(np.expand_dims(x_val[0],0)), axis=1)[0])

<h3>Evolutionary attack</h3>

In [71]:
from tqdm import tqdm

In [72]:
importlib.reload(EvoStrategyUniformProbs)
perturbed_images = 0
adv_evo_strategy = {}
for index in tqdm(range(0,50)):
#     print()
#     print(index)
    img = x_val_raw_sample[index]
#     print(img)
    label = np.argmax(y_val_one_hot_sample[index])
    adv_evo_strategy[index] = EvoStrategyUniformProbs.AdversarialPerturbationEvoStraegy(
        model=model,
        img=img,
        label=label,
        generation_size=15, 
        one_step_perturbation_pixel_count=1,
        verbose=False,
        zero_one_scale=False,
        range_scale_int=True,
#         max_rand=int(x_val.max()),
#         min_rand=int(x_val.min())
        preprocess = preprocess_input
    )
    no_steps = adv_evo_strategy[index].run_adversarial_attack(steps=1000)
    if adv_evo_strategy[index].is_perturbed() and no_steps > 0:
        perturbed_images += 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [05:42<00:00,  6.84s/it]


In [84]:
np.shape(x_val_raw_sample[0])

(224, 224, 3)

In [89]:
idx=10
total_l2 = 0
for idx in range(50):
    curr_l2 = np.sqrt(np.sum((adv_evo_strategy[idx].get_best_candidate() - x_val_raw_sample[idx]) ** 2)) / 255
    total_l2 += curr_l2
    print(curr_l2)

3.423331705729167
4.000403789445466
5.115678615196078
10.842918007046569
7.583658854166667
8.778058938419118
0.0
7.656060910692402
4.578708065257353
0.0
2.845589192708333
0.0
9.276169960171568
0.0
5.413813093596814
0.0
8.085151462928922
11.758141850490196
2.5397750555300247
2.5759959501378678
0.0
8.175011488970588
0.0
5.270325425091912
5.366815544577205
5.756228458180147
4.312252987132353
0.0
12.049950214460784
5.537902113970588
0.0
7.907203105851716
6.325018669577206
3.061740770526961
0.0
5.830818684895833
17.162591911764707
0.0
15.814234834558823
0.0
0.0
0.0
11.032848881740197
8.475209673713236
0.0
0.0
5.02354616651348
1.8776206820618873
5.341319125306373
4.045876895680147


In [91]:
total_l2 / 50 / (224 * 224)

9.280929969949405e-05

In [73]:
evoba_stats = utils.get_evoba_stats(adv_evo_strategy)

In [74]:
utils.print_evoba_stats(evoba_stats)


EvoBA STATS (L0 attack)
____________________
Perturbed successfully 50/50 images
Average query count: 1198.3
Average l0 distance: 238.26

Median query count: 496.0
Median l0 dist: 98.5

Max query count: 9211
Max l0 dist: 1822
____________________



In [ ]:
# importlib.reload(EvoStrategy)
# # perturbed_images = 0
# # adv_evo_strategy = {}
# for index in range(50,100):
#     print()
#     print(index)
#     img = x_val_raw_sample[index]
# #     print(img)
#     label = np.argmax(y_val_one_hot_sample[index])
#     adv_evo_strategy[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(
#         model=model,
#         img=img,
#         label=label,
#         generation_size=20, 
#         one_step_perturbation_pixel_count=1,
#         verbose=True,
#         zero_one_scale=False,
#         range_scale_int=True,
# #         max_rand=int(x_val.max()),
# #         min_rand=int(x_val.min())
#         preprocess = preprocess_input
#     )
#     no_steps = adv_evo_strategy[index].run_adversarial_attack(steps=1000)
#     if adv_evo_strategy[index].stop_criterion() and no_steps > 0:
#         perturbed_images += 1

In [ ]:
for index in range(50):
    if not adv_evo_strategy[index].stop_criterion():
        print(index)

In [ ]:
count_q = 0
sum_q = 0
query_list = []
fair_indices = []
for index in range(50):
    if(adv_evo_strategy[index].queries > 1):
        count_q +=1
        sum_q += adv_evo_strategy[index].queries
        query_list.append(adv_evo_strategy[index].queries)
        fair_indices.append(index)
        
print("Average queries for EVO algo:", sum_q/count_q)
print("Total count of perturbed images (classified correctly initially):", perturbed_images)

In [ ]:
np.median(query_list)

In [ ]:
np.mean(query_list)

In [ ]:
plt.hist(query_list)

In [ ]:
plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(12,8))
plt.title("Query histogram for EvoBA(1, 15) on ImageNet", fontdict={"size":22})
plt.hist(query_list)
plt.show()

In [ ]:
import math
from tqdm import tqdm
l0_dists = []
for index_diff in tqdm(fair_indices):
    diff = np.abs(adv_evo_strategy[index_diff].get_best_candidate() - x_val_raw_sample[index_diff])
#     diff = np.reshape(diff, (32, 32, 3))
    diff = (diff!=0)
    l0_dist = np.sum(diff)
    l0_dists.append(l0_dist)
#     print("L2 distance:", math.sqrt(np.sum(np.reshape(diff, (-1))**2)))
#     plt.imshow(np.reshape(adv_evo_strategy[index_diff].get_best_candidate(), (28, 28)))
#     plt.show()
#     print("Prediction:", model.predict(np.array([adv_evo_strategy[index_diff].get_best_candidate()])))

In [ ]:
np.median(l0_dists)

In [ ]:
plt.hist(queries)

In [ ]:
np.median(l2_dists)/255

In [ ]:
plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(12,8))
plt.title("L0 histogram for EvoBA(1, 15) on ImageNet", fontdict={"size":22})
plt.hist(l0_dists)
plt.show()

In [ ]:
import math
l2_dists = []
for index_diff in tqdm(fair_indices):
    diff = np.abs(adv_evo_strategy[index_diff].get_best_candidate() - x_val_raw_sample[index_diff])
#     diff = np.reshape(diff, (32, 32, 3))
    l2_dist = math.sqrt(np.sum(np.reshape(diff, (-1))**2))
    l2_dists.append(l2_dist)
#     print("L2 distance:", math.sqrt(np.sum(np.reshape(diff, (-1))**2)))
#     plt.imshow(np.reshape(adv_evo_strategy[index_diff].get_best_candidate(), (28, 28)))
#     plt.show()
#     print("Prediction:", model.predict(np.array([adv_evo_strategy[index_diff].get_best_candidate()])))

In [ ]:
np.shape(x_val_sample[0])

In [ ]:
np.mean(l2_dists)/255

In [ ]:
np.mean(l2_dists)/(255 * 224 * 224 * 3)

In [ ]:
w=10
h=10
fig=plt.figure(figsize=(20, 9))
fig.tight_layout()
plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.1, wspace=0.2)

columns = 5
rows = 2
img_curr = 0
for i in range(1, columns + 1):
    img_indx = fair_indices[imgss[img_curr]]
    initial = (keras_idx_to_name[adv_evo_strategy[img_indx].label])
    if len(initial) > 30:
        initial = initial[:21] + "\n" + initial[21:]
    fig.add_subplot(rows, columns, i)
    plt.title(f"Original, \n {initial}", fontdict={"size":18})
    img_start = adv_evo_strategy[img_indx].img.astype(int)
    plt.imshow(img_start)
    
    fig.add_subplot(rows, columns, i + 5)
    
    img_final = adv_evo_strategy[img_indx].get_best_candidate().astype(int)
    predss = model.predict(np.expand_dims(img_final, axis=0))[0]
    predicted = np.argmax(predss)
    diff = math.sqrt(np.sum((img_final - img_start) **2))
    l2_distance = int((diff/(255)) * 100)/100
    l0_distance = (img_start != img_final).sum()
    final = (keras_idx_to_name[predicted])
    plt.title(f"Perturbed, \n {final} L2 distance:{l2_distance}\n L0 distance:{l0_distance}", fontdict={"size":18})
    plt.imshow(img_final)
    img_curr += 1
plt.show()

In [ ]:
for index_diff in tqdm(np.array(fair_indices)[imgss]):
    print(index_diff)
    initial = (keras_idx_to_name[adv_evo_strategy[index_diff].label])
    img = adv_evo_strategy[index_diff].get_best_candidate().astype(int)
    plt.imshow(img.astype(int))
    predicted = (np.argmax(model.predict(np.expand_dims(img, axis=0))[0]))
    final = (keras_idx_to_name[predicted])
    plt.show()

In [ ]:
importlib.reload(EvoStrategy)
from tqdm import tqdm
perturbed_images_bf = 0
adv_evo_strategy_bf = {}
for index in tqdm(range(len(x_val_sample))):
    if index % 10 == 0:
        verbose = True
    else:
        verbose = False
    img = x_val_sample[index]
    label = np.argmax(y_val_one_hot_sample[index])
    adv_evo_strategy_bf[index] = EvoStrategy.AdversarialPerturbationBFStraegy(
        model=model,
        img=img,
        label=label,
        generation_size=20, 
        one_step_perturbation_pixel_count=5,
        verbose=verbose
    )
    no_steps_bf = adv_evo_strategy_bf[index].run_adversarial_attack(steps=100)
    if adv_evo_strategy_bf[index].stop_criterion() and no_steps_bf > 0:
        perturbed_images_bf += 1
    adv_evo_strategy_bf[index].active_generation = []
    adv_evo_strategy_bf[index].fitness_scores = []
    gc.collect()

In [ ]:
count_q_bf = 0
sum_q_bf = 0
for index in tqdm(range(len(x_val_sample))):
    if(adv_evo_strategy_bf[index].queries > 1):
        count_q_bf +=1
        sum_q_bf += adv_evo_strategy_bf[index].queries
        
print("Average queries for EVO algo:", sum_q_bf/count_q_bf)
print("Total count of perturbed images (classified correctly initially):", perturbed_images_bf)

In [ ]:
671/len(x_val_sample)

In [ ]:
import math
from tqdm import tqdm
l2_dists = []
for index_diff in tqdm(range(50)):
    diff = np.abs(adv_evo_strategy[index_diff].get_best_candidate() - x_val_sample[index_diff])
#     diff = np.reshape(diff, (32, 32, 3))
    l2_dist = math.sqrt(np.sum(np.reshape(diff, (-1))**2))
    l2_dists.append(l2_dist)
#     print("L2 distance:", math.sqrt(np.sum(np.reshape(diff, (-1))**2)))
#     plt.imshow(np.reshape(adv_evo_strategy[index_diff].get_best_candidate(), (28, 28)))
#     plt.show()
#     print("Prediction:", model.predict(np.array([adv_evo_strategy[index_diff].get_best_candidate()])))

In [ ]:
np.shape(x_val_sample[0])

In [ ]:
np.mean(l2_dists)/(255 * 224 * 224)

In [ ]:
importlib.reload(EvoStrategy)
# TODO: add per channel perturbation, verify that it succeeds faster
# TODO: momentum approach
perturbed_images = 0
adv_evo_strategy = {}
for index in range(100):
    print()
    print(index)
    img = x_val_sample[index]
    label = np.argmax(y_val_one_hot_sample[index])
    adv_evo_strategy[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(
        model=model,
        img=img,
        label=label,
        generation_size=20, 
        one_step_perturbation_pixel_count=10
    )
    no_steps = adv_evo_strategy[index].run_adversarial_attack(steps=50)
    if adv_evo_strategy[index].stop_criterion() and no_steps > 0:
        perturbed_images += 1

In [ ]:
count_q = 0
sum_q = 0
for index in range(0,100):
    if(adv_evo_strategy[index].queries > 1):
        count_q +=1
        sum_q += adv_evo_strategy[index].queries
        
print("Average queries for EVO algo:", sum_q/count_q)
print("Total count of perturbed images (classified correctly initially):", perturbed_images)

In [ ]:
importlib.reload(EvoStrategy)
import gc
# TODO: add per channel perturbation, verify that it succeeds faster
# TODO: momentum approach
perturbed_images = 0
adv_evo_strategy = {}
for index in range(100):
    print()
    print(index)
    img = x_val_sample[index]
    label = np.argmax(y_val_one_hot_sample[index])
    adv_evo_strategy[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(
        model=model,
        img=img,
        label=label,
        generation_size=50, 
        one_step_perturbation_pixel_count=10
    )
    no_steps = adv_evo_strategy[index].run_adversarial_attack(steps=40)
    if adv_evo_strategy[index].stop_criterion() and no_steps > 0:
        perturbed_images += 1
    gc.collect()

In [ ]:
count_q = 0
sum_q = 0
for index in range(0,100):
    if(adv_evo_strategy[index].queries > 1):
        count_q +=1
        sum_q += adv_evo_strategy[index].queries
        
print("Average queries for EVO algo:", sum_q/count_q)
print("Total count of perturbed images (classified correctly initially):", perturbed_images)

In [ ]:
print("HERE")

In [ ]:
importlib.reload(SimbaWrapper)
simba_wrapper = SimbaWrapper.SimbaWrapper(model, x_val_sample, y_val_one_hot_sample, 0.1, max_queries=2000, max_l0_distance=150)

In [ ]:
simba_wrapper.run_simba()

In [ ]:
simba_wrapper.X_modified[0].min()

In [ ]:
x_val_sample[0].min()